In [1]:
import torch
import torchinfo
from torchvision.models import mobilenet_v2
from torch.utils.data import DataLoader

C:\Users\Prolay\anaconda3\envs\torch_2.1.0\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

from level_1_dataloader import image_dataloader
from model.level_1_classifier import level_1_classifier, level_1_model, level_1_output_layer
from model.level_2_classifier import level_2_classifier, level_2_output_layer, level_2_pre_model_concate, level_2_post_model
from Trainer_1 import Trainer_level_1, Trainer_level_2
from Tester_1 import Tester_level_1, Tester_level_2

import copy

In [3]:
train_set = image_dataloader(csv_file = 'train_set.csv', root_dir = 'coverpage/coverpage/')
val_set = image_dataloader(csv_file = 'val_set.csv', root_dir = 'coverpage/coverpage/')
test_set = image_dataloader(csv_file = 'test_set.csv', root_dir = 'coverpage/coverpage/')

In [4]:
print('no. of training sample',len(train_set))
print('no. of validation sample',len(val_set))
print('no. of testing sample',len(test_set))

no. of training sample 22340
no. of validation sample 2804
no. of testing sample 2809


In [5]:

train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
print(len(train_dataloader))

val_dataloader = DataLoader(val_set, batch_size=32, shuffle=True)
print(len(val_dataloader))

test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)
print(len(test_dataloader))

699
88
88


In [6]:
import torch
from torchvision.models import mobilenet_v2
import torchinfo

def fine_tune_load_model():
    # Load the pretrained MobileNetV2 model
    model = mobilenet_v2(weights="IMAGENET1K_V1")
    
    # Remove the top classification layer
    modules = list(model.children())[:-1]
    new_model = torch.nn.Sequential(*modules)
    
    # Add a Flatten layer to the new model
    new_model.add_module("flatten", torch.nn.Flatten(start_dim=1))
    
    return new_model

# Load the fine-tuned MobileNetV2 model
FE_model = fine_tune_load_model()



In [8]:
FE_model = fine_tune_load_model()
print(FE_model)

Sequential(
  (0): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, m

In [10]:
torchinfo.summary(FE_model, input_size=(3, 224, 224), batch_dim=0, col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose=1)


Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
Sequential                                         [1, 3, 224, 224]          [1, 62720]                --                        --                        --
├─Sequential: 1-1                                  [1, 3, 224, 224]          [1, 1280, 7, 7]           --                        --                        --
│    └─Conv2dNormActivation: 2-1                   [1, 3, 224, 224]          [1, 32, 112, 112]         --                        --                        --
│    │    └─Conv2d: 3-1                            [1, 3, 224, 224]          [1, 32, 112, 112]         864                       [3, 3]                    10,838,016
│    │    └─BatchNorm2d: 3-2                       [1, 32, 112, 112]         [1, 32, 112, 112]         64                        --                        64
│    │    └─ReLU6: 3-3               

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
Sequential                                         [1, 3, 224, 224]          [1, 62720]                --                        --                        --
├─Sequential: 1-1                                  [1, 3, 224, 224]          [1, 1280, 7, 7]           --                        --                        --
│    └─Conv2dNormActivation: 2-1                   [1, 3, 224, 224]          [1, 32, 112, 112]         --                        --                        --
│    │    └─Conv2d: 3-1                            [1, 3, 224, 224]          [1, 32, 112, 112]         864                       [3, 3]                    10,838,016
│    │    └─BatchNorm2d: 3-2                       [1, 32, 112, 112]         [1, 32, 112, 112]         64                        --                        64
│    │    └─ReLU6: 3-3               